# 导入模块

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")
from jsonpath import jsonpath
import pymysql
from sqlalchemy import create_engine
import json
import random
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 链接数据库

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 查询数据

In [3]:
sql = '''
select
om.id as order_id,
om.create_time, om.order_number, om.status, tprm.all_deposit, tprm.all_rental,tprm.purchase_amount, tod.new_actual_money, tod.sku_attributes, tpmn.name 机型,
top.total_describes, pa.name activity_name,tmu.true_name, tmu.id_card_num, om.user_mobile
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,tps.deposit, tojo.app_type
from db_digua_business.t_order om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
-- 商品表                 
left join db_digua_business.t_product tp on tp.id = tod.product_id
left join db_digua_business.t_product_sku tps on tps.id=tod.sku_id
-- 商品型号
left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top on om.id = top.order_id 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 京东外部订单关联表
left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id
where  date_format(om.create_time, '%Y-%m-%d')>='2025-06-01' 
and date_format(om.create_time, '%Y-%m-%d')<='2025-07-07' 
and om.order_type='JD_ORDER'
'''
df = query(sql)

# 清理数据

In [7]:
# 处理日期
df["下单日期"]=df["create_time"].dt.date
# df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')


dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

df.loc[:,"内存"]=df.apply(lambda x: clean.getneicun(x["sku_attributes"]),axis=1)
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
# df

,order_id,create_time,order_number,status,all_deposit,all_rental,purchase_amount,new_actual_money,sku_attributes,机型,total_describes,activity_name,true_name,id_card_num,user_mobile,status2,deposit,app_type,下单日期,下单月份,状态编码,内存,租赁方案,进件,出库
0,2782446,2025-06-01 00:21:05,A2025060100205144,13,NaN,NaN,NaN,5416.7,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",小米15,None,京东活动,褚清印,411322200704112915,17530657296,订单取消,4199.0,JR_APP,2025-06-01,2025-06,1,12GB+256GB,租完归还,0,0
1,2782497,2025-06-01 00:43:28,A2025060100430894,13,NaN,NaN,NaN,10576.7,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",华为Pura70Pro+,None,京东活动,邓鸿滔,350403198510223018,13459834333,订单取消,8199.0,JR_APP,2025-06-01,2025-06,1,16GB+1TB,租完归还,0,0
2,2782527,2025-06-01 00:55:46,A20250601005526125,13,NaN,NaN,NaN,6061.7,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",小米15,None,京东活动,伍湘候,433123199311201832,15576953110,订单取消,4699.0,JR_APP,2025-06-01,2025-06,1,16GB+512GB,租完归还,0,0
3,2782539,2025-06-01 00:59:19,A20250601005906138,13,NaN,NaN,NaN,9286.7,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",华为Pura70Pro,"[""系统自动操作审核订单A20250601005906138，审核结果：不通过,拒绝理由：命...",京东活动,买尔旦江·阿吾提,653126199903171219,15199114520,订单取消,7199.0,JR_APP,2025-06-01,2025-06,1,12GB+1TB,租完归还,0,0
4,2782540,2025-06-01 00:59:25,A20250601005903137,13,NaN,NaN,NaN,8641.7,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",华为Mate 70 Pro,None,京东活动,崔铁柱,130703198302221531,13294037177,订单取消,6699.0,JR_APP,2025-06-01,2025-06,1,12GB+512GB,租完归还,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48319,3156577,2025-07-07 19:22:03,A20250707192150291,4,9999.0,5899.57,8440.0,12898.7,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",iPhone16ProMax,"[""修改收货信息：将原收货信息 秦小姐 180****3337 江苏省 无锡市 惠山区 华新...",京东活动,秦丽恬,320283199706014226,18006193337,租赁中,9999.0,JD_APP,2025-07-07,2025-07,7,256GB,租完归还,1,1
48421,3157260,2025-07-07 20:33:21,A20250707203310283,4,9999.0,5899.57,8440.0,12898.7,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",iPhone16ProMax,"[""88888_林思慧订单A20250707203310283，msg=待客户充值，1+1+...",京东活动,裴杨洋,421087200007158225,17671935206,租赁中,9999.0,JD_APP,2025-07-07,2025-07,7,256GB,租完归还,1,1
48424,3157280,2025-07-07 20:37:30,A20250707203717305,4,8999.0,5759.51,7410.0,11608.7,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",iPhone16Pro,"[""系统已发起自动扣款，扣款期数：1+1（注：自动扣款无法保证最终一定成功请注意检查）"", ...",京东活动,倪梦琪,532801199310284528,13368858813,租赁中,8999.0,JD_APP,2025-07-07,2025-07,7,256GB,租完归还,1,1
48482,3157797,2025-07-07 21:54:19,A20250707215353362,4,9999.0,5899.57,8440.0,12898.7,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",iPhone16ProMax,"[""修改收货信息：将原收货信息 陶德天 155****2547 吉林省 白山市 临江市 兴隆...",京东活动,陶德天,220623199606130328,15584922547,租赁中,9999.0,JD_APP,2025-07-07,2025-07,7,256GB,租完归还,1,1


In [8]:
def zh(df):
    df.loc[:, '进件'] = np.where(df.status.isin([1,13]), 0, 1)
    df.loc[:, '出库'] = np.where(df.status.isin([2,3,4,5,6,8,15]), 1, 0)
    df_group = df.groupby('下单日期').agg({'order_number': 'count', '进件': 'sum', '出库': 'sum'}).rename(columns={'order_number': '去重订单数'}).T
    df_group.loc[:, '月合计'] = df_group.sum(axis=1)
    df_group.insert(15, '15日合计', df_group.iloc[:, :15].sum(axis=1))
    df_group = df_group.T
    df_group.loc[:, '进件出库率'] = (df_group.出库/df_group.进件).map(lambda x: format(x, '.2%'))
    df_group.loc[:, '订单出库率'] = (df_group.出库/df_group.去重订单数).map(lambda x: format(x, '.2%'))
    df_groups = df_group.T
    return df_groups
df_all = zh(df)
df_all.loc['京东商城',:] = np.nan
df_jd = df[df.app_type=='JD_APP']
df_jd_all = zh(df_jd)
df_jd_all.loc['京东金融',:] = np.nan
df_fjd = df[df.app_type!='JD_APP']
df_fjd_all = zh(df_fjd)
df_groups = pd.concat([df_all, df_jd_all, df_fjd_all])
# df_groups

下单日期,2025-06-01,2025-06-02,2025-06-03,2025-06-04,2025-06-05,2025-06-06,2025-06-07,2025-06-08,2025-06-09,2025-06-10,2025-06-11,2025-06-12,2025-06-13,2025-06-14,2025-06-15,15日合计,2025-06-16,2025-06-17,2025-06-18,2025-06-19,2025-06-20,2025-06-21,2025-06-22,2025-06-23,2025-06-24,2025-06-25,2025-06-26,2025-06-27,2025-06-28,2025-06-29,2025-06-30,2025-07-01,2025-07-02,2025-07-03,2025-07-04,2025-07-05,2025-07-06,2025-07-07,月合计
去重订单数,132,148,150,215,341,454,533,551,1338,869,756,729,930,966,810,8922,1114,1196,1689,1308,1147,985,935,1286,1220,1385,1363,1228,967,899,1228,1382,1425,1438,1512,1294,1481,1651,37055
进件,77,99,108,134,224,309,359,363,906,580,480,462,614,607,529,5851,703,751,1070,831,744,649,603,854,775,911,858,789,599,550,781,901,906,1009,1029,858,956,1070,24048
出库,1,4,2,2,8,5,7,7,14,25,21,18,30,31,24,199,35,45,47,47,31,38,38,57,42,37,36,33,18,22,36,44,42,44,57,44,39,43,1074
进件出库率,1.30%,4.04%,1.85%,1.49%,3.57%,1.62%,1.95%,1.93%,1.55%,4.31%,4.38%,3.90%,4.89%,5.11%,4.54%,3.40%,4.98%,5.99%,4.39%,5.66%,4.17%,5.86%,6.30%,6.67%,5.42%,4.06%,4.20%,4.18%,3.01%,4.00%,4.61%,4.88%,4.64%,4.36%,5.54%,5.13%,4.08%,4.02%,4.47%
订单出库率,0.76%,2.70%,1.33%,0.93%,2.35%,1.10%,1.31%,1.27%,1.05%,2.88%,2.78%,2.47%,3.23%,3.21%,2.96%,2.23%,3.14%,3.76%,2.78%,3.59%,2.70%,3.86%,4.06%,4.43%,3.44%,2.67%,2.64%,2.69%,1.86%,2.45%,2.93%,3.18%,2.95%,3.06%,3.77%,3.40%,2.63%,2.60%,2.90%
京东商城,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
去重订单数,NaN,NaN,NaN,NaN,53,66,73,97,174,121,103,93,159,144,93,2121,166,249,264,266,272,179,158,305,340,584,523,518,387,341,462,543,525,615,647,541,464,655,10180
进件,NaN,NaN,NaN,NaN,40,45,53,60,127,104,102,91,152,136,93,1917,158,243,253,260,265,176,151,294,327,555,501,485,370,318,452,516,509,588,617,517,447,630,9635
出库,NaN,NaN,NaN,NaN,1,0,2,1,4,4,7,7,15,14,9,124,6,18,19,17,18,11,12,18,23,22,14,17,8,13,11,18,22,24,35,27,23,25,465
进件出库率,NaN,NaN,NaN,NaN,2.50%,0.00%,3.77%,1.67%,3.15%,3.85%,6.86%,7.69%,9.87%,10.29%,9.68%,6.47%,3.80%,7.41%,7.51%,6.54%,6.79%,6.25%,7.95%,6.12%,7.03%,3.96%,2.79%,3.51%,2.16%,4.09%,2.43%,3.49%,4.32%,4.08%,5.67%,5.22%,5.15%,3.97%,4.83%


In [9]:
df_new = df[(df.出库==1)&(df.purchase_amount>0)]

df_group = df_new.groupby(['下单月份', '租赁方案']).agg({'new_actual_money': 'mean', 'all_rental': 'mean', 'deposit': 'mean', 'purchase_amount': 'mean'}).rename(columns={'new_actual_money': '签约价均值', 'all_rental': '总租金均值', 'deposit': '押金均值', 'purchase_amount': '采购金额均值'})
col_list = df_group.columns.to_list()
for col in col_list:
    df_group.loc[:, f'{col}'] = np.round(df_group[f'{col}'], 0)
df_group.loc[:, '签约价租售比'] = (df_group.签约价均值/df_group.押金均值).map(lambda x: format(x, '.2%'))
df_group.loc[:, '总租金租售比'] = (df_group.总租金均值/df_group.押金均值).map(lambda x: format(x, '.2%'))

# df_group

签约价均值    总租金均值    押金均值  采购金额均值   签约价租售比   总租金租售比
下单月份    租赁方案                                                    
2025-06 租完即送  12254.0  12254.0  9499.0  7913.0  129.00%  129.00%
        租完归还  12441.0   5775.0  9644.0  8015.0  129.00%   59.88%
2025-07 租完即送  12899.0  12899.0  9999.0  8375.0  129.00%  129.00%
        租完归还  12536.0   5777.0  9718.0  8129.0  129.00%   59.45%

In [67]:
df.loc[:, '机型内存'] = df.机型+'_'+df.内存
def sku(df, key):
    df_new = df[df.purchase_amount>0]
    df_group = df.groupby(f'{key}').agg({'进件': 'sum', '出库': 'sum'})
    df_new_group = df_new.groupby(f'{key}').agg({'new_actual_money': 'mean', 'all_rental': 'mean', 'deposit': 'mean', 'purchase_amount': 'mean'}).rename(columns={'new_actual_money': '签约价均值', 'all_rental': '总租金均值', 'deposit': '押金均值', 'purchase_amount': '采购金额均值'})
    df_group = df_group.merge(df_new_group, on=f'{key}', how='left')
    df_group = df_group.fillna(0)
    df_group.loc['汇总', :] = df_group.sum(axis=0)
    df_group.loc[:, '机型占比'] = (df_group.出库/df_group.loc['汇总', '出库']).map(lambda x: format(x, '.2%'))
    df_group.loc[:, '签约价租售比'] = (df_group.签约价均值/df_group.押金均值).map(lambda x: format(x, '.2%'))
    df_group.loc[:, '总租金租售比'] = (df_group.总租金均值/df_group.押金均值).map(lambda x: format(x, '.2%'))
    df_group = df_group.sort_values(by='进件', ascending=False)
    return df_group
df_sku = sku(df, '机型内存')
df_spu = sku(df, '机型')

In [68]:
with pd.ExcelWriter('F:/需求/瑞彤需求/京东复盘转化数据.xlsx', engine='xlsxwriter') as writer:
    df_groups.to_excel(writer, sheet_name='京东目标转化率')
    df_group.to_excel(writer, sheet_name='京东租售比')
    df_sku.to_excel(writer, sheet_name='京东sku')
    df_spu.to_excel(writer, sheet_name='京东spu')
